In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip3 install torch
!pip3 install transformers


     |████████████████████████████████| 317kB 2.8MB/s 
     |████████████████████████████████| 645kB 47.7MB/s 
     |████████████████████████████████| 1.0MB 43.3MB/s 
     |████████████████████████████████| 860kB 39.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=4a28ac46e556b9442be43761222ba3391cb08b38017c186825ff4bb1284fcdd0
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [3]:
import torch
from transformers import BertTokenizer, BertModel, BertForTokenClassification, BertConfig
from keras.preprocessing.sequence import pad_sequences
import sys
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import pandas as pd
import numpy as np
from itertools import combinations_with_replacement, permutations
import datetime

Using TensorFlow backend.


In [4]:
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
cuda0 = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(cuda0)

cuda


In [5]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=02ed252e5489d3248c00fa89dc0ad3e040f6a4050fe72f66c6edc66aad64a4b6
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [6]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.6 GB  | Proc size: 501.0 MB
GPU RAM Free: 11430MB | Used: 11MB | Util   0% | Total 11441MB


In [0]:
def process_data(path):
    first = True
    file_table = []
    with open(path) as fp:
        i = 0
        for line in fp:
            if not first:
                new_entry = []
                split_line = line.split()
                new_entry.append(split_line[0])
                new_entry.append(split_line[1])
                sep = ' '
                hexamers = list(map(lambda x: x.replace(']', '').replace('[', '').replace(',','').replace('\'', ''), split_line[2:1002]))
                if '--PAD--' in hexamers:
                    hexamers = hexamers[:hexamers.index('--PAD--')]
                hexamers_string = sep.join(hexamers)
                new_entry.append(hexamers_string)
                new_entry.append(list(map(lambda x: int(x.replace(']', '').replace('[', '').replace(',','').replace('.0','').replace('\'','')), split_line[1002:])))
                file_table.append(new_entry)
                i += 1
                if i == 500:
                  break
            first = False
    
    return handle_tokenization(file_table)

def handle_tokenization(file_table):
    MAX_LEN = 1002
    fourmers_path = '/content/drive/My Drive/k_mers/fourmers.txt'
    hexamers_path = '/content/drive/My Drive/k_mers/hexamers.txt'
    octamers_path = '/content/drive/My Drive/k_mers/octamers.txt'

    formatted_hexamers = ['[CLS] ' + f[2] + ' [SEP]' for f in file_table]
    labels = [[0] + f[3] + [0] for f in file_table]
    tokenizer = BertTokenizer(hexamers_path, max_len=MAX_LEN)
    #hexamer_set = generate_hexamer_tokens()
    #tokenizer.add_tokens(hexamer_set)
    attention_masks = [np.concatenate([np.ones((len(a.split()))), np.zeros((MAX_LEN - len(a.split())))]) for a in formatted_hexamers]
    
    return tokenizer, formatted_hexamers, attention_masks, labels


def generate_hexamer_tokens():
    nucleotides = ['A', 'T', 'G', 'C']
    hexamer_tuples = combinations_with_replacement(nucleotides, 6)
    hexamer_tokens = [''.join(hexamer_tuple) for hexamer_tuple in hexamer_tuples]
    hexamer_set = set()
    for token in hexamer_tokens:
        permuted_hexamers = [''.join(l) for l in list(permutations(token))]
        hexamer_set.update(permuted_hexamers)
    assert len(hexamer_set) == 4096
    return list(hexamer_set)

In [0]:
batch_size = 1
MAX_LEN = 1002

tokenizer, formatted_hexamers, attention_masks, labels = process_data('/content/drive/My Drive/split_samples_6-mer.txt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0) 

model = BertForTokenClassification(BertConfig.from_json_file('/content/drive/My Drive/bert_configuration.json'))
model.resize_token_embeddings(len(tokenizer))
model.to(device)

#model.load_state_dict(torch.load("/content/drive/My Drive/bert_splice_weights.pt"))
last_i = 0
with open('/content/drive/My Drive/bert_last_i.txt', 'r') as last_i_file:
  i = last_i_file.read()
  last_i = int(i)

print(last_i)
optimizer = Adam(model.parameters(), lr=3e-5)

training_loss = []
model.train()
class_weights = torch.tensor(np.array([1.0, 100.0])).float().cuda()
loss = CrossEntropyLoss(weight=class_weights)
print("ready to train")
total_loss = 0
for batch in range(len(formatted_hexamers) // batch_size):
    seq_batch = formatted_hexamers[batch*batch_size:(batch+1)*batch_size]
    attention_masks_batch = attention_masks[batch*batch_size:(batch+1)*batch_size]
    attention_masks_batch = torch.tensor(attention_masks_batch).long().cuda()
    split_seq_batch = seq_batch[0].split()
    batch_ids = [tokenizer.convert_tokens_to_ids(split_seq_batch)]
    batch_ids = pad_sequences(batch_ids, maxlen=MAX_LEN, truncating='post', padding='post')[0]
    batch_ids = torch.tensor(batch_ids).unsqueeze(0).long().cuda()
    batch_labels = torch.tensor(labels[batch*batch_size:(batch+1)*batch_size]).long().cuda()

    optimizer.zero_grad()
    predictions = model(batch_ids, attention_mask=attention_masks_batch)
    l = loss(predictions[0].squeeze(0), batch_labels[0])
    total_loss += l.item()
    training_loss.append(l.item())
    l.backward()
    optimizer.step()
    if batch > 0 and batch % 500 == 0:
      path = "/content/drive/My Drive/bert_splice_weights.pt"
      last_seq_path = "/content/drive/My Drive/bert_last_i.txt"
      with open(last_seq_path, 'w+') as seq_record:
        seq_record.write(str(batch))
      loss_trace_path = "/content/drive/My Drive/bert_loss_trace.txt"
      with open(loss_trace_path, 'a+') as loss_record:
        loss_record.write(str(batch) + ' loss: ' + str(total_loss/100) + '\n')
      torch.save(model.state_dict(), path)
    if batch > 0 and batch % 100 == 0:
      print("Batch: " + str(batch) + " loss: " + str(total_loss / 100))
      total_loss = 0




Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors


ready to train


Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Batch: 100 loss: 0.8228263034112752


Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (547 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

...

Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Batch: 400 loss: 0.6378443499654531


Token indices sequence length is longer than the specified maximum sequence length for this model (892 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Batch: 500 loss: 0.5276786737889051


Token indices sequence length is longer than the specified maximum sequence length for this model (917 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1002 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for